In [23]:
import numpy as np

In [30]:
with open('table1.typ', 'wt') as file:
    out = file

    print('#table(columns: 8, ', file=out)
    print('[$i$],[$x$],[$y$],[$z$],[$K$],[$l$],[$Delta y$],[$Delta z$],', file=out)

    h = 0.1
    XS = np.linspace(0, 0.3, 4)
    N = XS.shape[0]

    YS = [2]

    # y' = z
    # z' = 2 y + x exp(-x)

    g = lambda x, y: 2 * y + x * np.exp(-x) # y''
    f = lambda z: z                      # y'

    y = 2
    z = 1
    i = 0
    for x in XS[:-1]:

        print(f"// X = {x:.2f}", file=out)
        
        x1 = x; y1 = y; z1 = z
        K1 = h * g(x1, y1)
        L1 = h * f(z1)

        x2 = x + h / 2; y2 = y + L1/2; z2 = z + K1/2
        K2 = h * g(x2, y2)
        L2 = h * f(z2)

        x3 = x + h / 2; y3 = y + L2/2; z3 = z + K2/2
        K3 = h * g(x3, y3)
        L3 = h * f(z3)

        x4 = x + h; y4 = y + L3/2; z4 = z + K3/2
        K4 = h * g(x4, y4)
        L4 = h * f(z4)

        print(f'[{i:1d}],', file=out)

        print('[{:4.2f}\\ {:4.2f}\\ {:4.2f}\\ {:4.2f}],'.format(x1, x2, x3, x4), file=out)
        print('[{:8.6f}\\ {:8.6f}\\ {:8.6f}\\ {:8.6f}],'.format(y1, y2, y3, y4), file=out)
        print('[{:8.6f}\\ {:8.6f}\\ {:8.6f}\\ {:8.6f}],'.format(z1, z2, z3, z4), file=out)
        print('[{:8.6f}\\ {:8.6f}\\ {:8.6f}\\ {:8.6f}],'.format(K1, K2, K3, K4), file=out)
        print('[{:8.6f}\\ {:8.6f}\\ {:8.6f}\\ {:8.6f}],'.format(L1, L2, L3, L4), file=out)

        print('[{:8.6f}\\ {:8.6f}\\ {:8.6f}\\ {:8.6f}],'.format(L1, 2*L2, 2*L3, L4), file=out)
        print('[{:8.6f}\\ {:8.6f}\\ {:8.6f}\\ {:8.6f}],'.format(K1, 2*K2, 2*K3, K4), file=out)
        print('[],'*6, f'[{sum([L1, 2*L2, 2*L3, L4]):8.6f}]', ',', sep='', file=out)
        print(f'[{sum([K1, 2*K2, 2*K3, K4]):8.6f}]', ',', sep='', file=out)

        i += 1
        z = z + 1/6 * (K1+2*K2+2*K3+K4)
        y = y + 1/6 * (L1+2*L2+2*L3+L4)

        YS.append(y)

    print(f'[{N-1}],[{XS[-1]:4.2f}],[{y:8.6f}],[{z:8.6f}],', '[],'*4, file=out)
    print(')', file=out)

In [25]:
import sympy as sp

symbX = sp.symbols('x')
symbY, symbZ = sp.symbols('y z', cls=sp.Function)

In [26]:
sol = sp.dsolve(
    sp.Eq(
        symbY(symbX).diff(symbX,2),
        2*symbY(symbX) + symbX * sp.exp(-symbX)
    ),
    ics={
        symbY(0): 2,
        symbY(symbX).diff(symbX,1).subs(symbX, 0): 1
    }
).rhs
sol

-x*exp(-x) + sqrt(2)*exp(sqrt(2)*x) - sqrt(2)*exp(-sqrt(2)*x) + 2*exp(-x)

In [27]:
YS_exact = list(map(
    lambda x: float(sol.subs(symbX, x)),
    XS
))
YS_exact

[2.0, 2.1205257615700868, 2.2844247702339073, 2.4957163672074647]

In [28]:
sol.subs(symbX, 0.3).n(8)

2.4957164

In [36]:
with open('table2.typ', 'wt') as out:
    print('#table(columns: 4, inset: (y: 0.6em),', file=out)
    print("""[$x_i$], 
    [метод\\ Рунге-Кутта\\ $y_i$],
    [Точное\\ решение\\ $y(x_i)$],
    [Абсолютная\\ погрешность\\ $|y_i - y(x_i)|$],""", file=out)

    for i in range(N):
        D = abs(YS[i] - YS_exact[i])
        print(f"[$x_{i}$],[{YS[i]:8.6f}],[{YS_exact[i]:8.6f}],[{D:8.6f}],", file=out)

    print(')', file=out)